In [4]:
from quickumls import QuickUMLS
import pandas as pd

In [2]:
# !python -m quickumls.install /secure/shared_data/UMLS/Destination/2024AA/META /home/yl3427/cylab/QuickUMLS

In [5]:
# all_types =  {f"T{str(i).zfill(3)}" for i in range(1, 1000)}
all_types = {
    'T020',
    'T052',
    'T100',
    'T087',
    'T116',
    'T011',
    'T190',
    'T017',
    'T008',
    'T195',
    'T194',
    'T007',
    'T053',
    'T038',
    'T123',
    'T091',
    'T122',
    'T012',
    'T029',
    'T023',
    'T030',
    'T031',
    'T022',
    'T088',
    'T025',
    'T026',
    'T043',
    'T049',
    'T103',
    'T120',
    'T104',
    'T185',
    'T201',
    'T200',
    'T077',
    'T019',
    'T056',
    'T060',
    'T047',
    'T203',
    'T065',
    'T196',
    'T018',
    'T071',
    'T069',
    'T126',
    'T204',
    'T051',
    'T050',
    'T099',
    'T033',
    'T013',
    'T168',
    'T021',
    'T169',
    'T004',
    'T028',
    'T045',
    'T083',
    'T064',
    'T096',
    'T102',
    'T131',
    'T058',
    'T093',
    'T125',
    'T016',
    'T068',
    'T078',
    'T129',
    'T130',
    'T055',
    'T037',
    'T197',
    'T170',
    'T034',
    'T059',
    'T171',
    'T066',
    'T015',
    'T073',
    'T074',
    'T048',
    'T041',
    'T063',
    'T044',
    'T085',
    'T070',
    'T191',
    'T114',
    'T086',
    'T090',
    'T057',
    'T042',
    'T109',
    'T001',
    'T032',
    'T040',
    'T092',
    'T046',
    'T101',
    'T121',
    'T067',
    'T072',
    'T039',
    'T002',
    'T098',
    'T097',
    'T094',
    'T080',
    'T081',
    'T192',
    'T089',
    'T014',
    'T062',
    'T075',
    'T095',
    'T184',
    'T054',
    'T082',
    'T167',
    'T079',
    'T061',
    'T024',
    'T010',
    'T005',
    'T127'
}


In [7]:
# matcher = QuickUMLS(quickumls_fp, overlapping_criteria, threshold,
#                     similarity_name, window, accepted_semtypes)
train_df = pd.read_csv('/home/yl3427/cylab/SOAP_MA/BioNLP2023-1A-Train.csv')
test_df = pd.read_csv('/home/yl3427/cylab/SOAP_MA/BioNLP2023-1A-Test.csv')
reference_df = pd.concat([train_df, test_df], ignore_index=True)
filtered_matcher = QuickUMLS('/home/yl3427/cylab/QuickUMLS', 
                    overlapping_criteria="length",
                    threshold=0.8,
                    # accepted_semtypes=all_types,
                    accepted_semtypes=['T184', 'T034', 'T046','T047', 'T048', 'T049', 'T190', 'T191', 'T037', 'T058', 'T059', 'T060', 'T061', 'T039', 'T040', 'T041', 'T042', 'T201', 'T203']
                    )
all_matcher  = QuickUMLS('/home/yl3427/cylab/QuickUMLS', 
                    overlapping_criteria="length",
                    threshold=0.8,
                    accepted_semtypes=all_types,
                    # accepted_semtypes=['T033', 'T184', 'T034', 'T046','T047', 'T048', 'T049', 'T190', 'T191', 'T037', 'T058', 'T059', 'T060', 'T061', 'T039', 'T040', 'T041', 'T042', 'T201', 'T203']
                    )

In [9]:
type_stat = {}
pool_of_terms = set()
valid_fileids = set()

for i, row in reference_df.iterrows():
    summary = row['Summary']
    new_summary = summary

    try:
        sum_lst= summary.split(";")
    except:
        continue

    matched_sum_lst = [all_matcher.match(s, best_match=True) for s in sum_lst]
    if not all(matched_sum_lst):
        continue

    for s in sum_lst:
        matches = all_matcher.match(s, best_match=True)
        print(matches[0])
        term = sorted(matches[0], key=lambda x: x['similarity'], reverse=True)[0]['term']
        new_summary = new_summary.replace(s, term)
        pool_of_terms.add(term)
        typ = sorted(matches[0], key=lambda x: x['similarity'], reverse=True)[0]['semtypes'].pop()
        if typ not in type_stat:
            type_stat[typ] = 0
        type_stat[typ] += 1
    
    print(f"file id: {row['File ID']}")
    print(f"Original: {summary}")
    print(f"New: {new_summary}")

    valid_fileids.add(row['File ID'])


       

[{'start': 2, 'end': 9, 'ngram': 'Hypoxia', 'term': 'Hypoxia', 'cui': 'C0242184', 'similarity': 1.0, 'semtypes': {'T046'}, 'preferred': 0}]
[{'start': 1, 'end': 13, 'ngram': 'Hyperkalemia', 'term': 'Hyperkalemia', 'cui': 'C0020461', 'similarity': 1.0, 'semtypes': {'T033'}, 'preferred': 0}]
file id: 188026.txt
Original: # Hypoxia:; Hyperkalemia
New: Hypoxia;Hyperkalemia
[{'start': 0, 'end': 18, 'ngram': 'PULMONARY EMBOLISM', 'term': 'Hx pulmonary embolism', 'cui': 'C0585968', 'similarity': 0.8421052631578947, 'semtypes': {'T033'}, 'preferred': 1}, {'start': 0, 'end': 18, 'ngram': 'PULMONARY EMBOLISM', 'term': 'Chr pulmonary embolism', 'cui': 'C0856722', 'similarity': 0.8, 'semtypes': {'T046'}, 'preferred': 1}, {'start': 0, 'end': 18, 'ngram': 'PULMONARY EMBOLISM', 'term': 'pulmonary embolism', 'cui': 'C0034065', 'similarity': 1.0, 'semtypes': {'T046'}, 'preferred': 0}]
[{'start': 1, 'end': 6, 'ngram': 'FEVER', 'term': 'fever', 'cui': 'C0015967', 'similarity': 1.0, 'semtypes': {'T184'}, 

In [13]:
len(valid_fileids), len(pool_of_terms)

(283, 423)

In [10]:
sorted(type_stat.items(), key=lambda x: x[1], reverse=True)

[('T047', 622),
 ('T033', 197),
 ('T046', 127),
 ('T184', 122),
 ('T048', 67),
 ('T191', 51),
 ('T061', 36),
 ('T201', 31),
 ('T169', 29),
 ('T002', 26),
 ('T079', 21),
 ('T080', 19),
 ('T034', 16),
 ('T170', 16),
 ('T037', 16),
 ('T082', 15),
 ('T081', 14),
 ('T023', 12),
 ('T121', 12),
 ('T030', 9),
 ('T078', 8),
 ('T031', 8),
 ('T007', 7),
 ('T204', 6),
 ('T011', 5),
 ('T060', 4),
 ('T058', 4),
 ('T074', 4),
 ('T090', 4),
 ('T038', 3),
 ('T029', 3),
 ('T168', 2),
 ('T020', 2),
 ('T190', 2),
 ('T049', 2),
 ('T055', 2),
 ('T185', 2),
 ('T052', 2),
 ('T129', 2),
 ('T043', 2),
 ('T019', 2),
 ('T025', 2),
 ('T042', 2),
 ('T041', 2),
 ('T116', 1),
 ('T005', 1),
 ('T077', 1),
 ('T059', 1),
 ('T122', 1),
 ('T102', 1),
 ('T101', 1),
 ('T073', 1)]

In [ ]:
one_match, multiple_match, no_match # for filtered semtypes

In [ ]:
one_match, multiple_match, no_match # for all semtypes

In [40]:
text = """67 y/o M CAD s/[**Initials (NamePattern4) **] [**Last Name (NamePattern4) **] [**Last Name (Prefixes) **] stent [**2169-11-15**] with recent cecum polypectomy
   who presents with bright red blood per rectum and hypotension following
   prep.
   ."""

In [ ]:
results = matcher.match(text, best_match=True)
type(results)
for result in results:
    print(result)

In [ ]:
matcher.match(text, best_match=False, ignore_syntax=True)

# now, start.

In [2]:
train_df = pd.read_csv('/home/yl3427/cylab/SOAP_MA/BioNLP2023-1A-Train.csv')
test_df = pd.read_csv('/home/yl3427/cylab/SOAP_MA/BioNLP2023-1A-Test.csv')
reference_df = pd.concat([train_df, test_df], ignore_index=True)
filtered_matcher = QuickUMLS('/home/yl3427/cylab/QuickUMLS', 
                    overlapping_criteria="length",
                    threshold=0.8,
                    accepted_semtypes=['T037', 'T046', 'T047', 'T048', 'T049', 'T190', 'T191'],
                    # accepted_semtypes=['T047']
                    )

In [16]:
all([0, "", [[]], "hello", "world", ""])

False

In [20]:
all(["he", [], "hello", "world"])

False

In [3]:
valid_fileids = set()
pool_of_terms = set()
reference_df['Summary2'] = reference_df['Summary'].apply(lambda x: "")
for i, row in reference_df.iterrows():
    summary = row['Summary']
    new_summary = summary

    try:
        sum_lst= summary.split(";")
    except:
        continue

    matched_sum_lst = [filtered_matcher.match(s, best_match=True) for s in sum_lst]
    if not all(matched_sum_lst):
        continue

    for s in sum_lst:
        matches = filtered_matcher.match(s, best_match=True)
        print(matches[0])
        term = sorted(matches[0], key=lambda x: x['similarity'], reverse=True)[0]['term']
        new_summary = new_summary.replace(s, term)
        pool_of_terms.add(term)

    reference_df.at[i, 'Summary2'] = new_summary
    valid_fileids.add(row['File ID'])


[{'start': 0, 'end': 11, 'ngram': 'Cholangitis', 'term': 'Cholangitis', 'cui': 'C0008311', 'similarity': 1.0, 'semtypes': {'T047'}, 'preferred': 0}]
[{'start': 11, 'end': 22, 'ngram': 'Heart Block', 'term': 'Heart Block', 'cui': 'C0018794', 'similarity': 1.0, 'semtypes': {'T047'}, 'preferred': 0}]
[{'start': 0, 'end': 21, 'ngram': 'Altered Mental Status', 'term': 'Altered Mental Status', 'cui': 'C0278061', 'similarity': 1.0, 'semtypes': {'T048'}, 'preferred': 1}]
[{'start': 0, 'end': 21, 'ngram': 'Altered mental status', 'term': 'Altered mental status', 'cui': 'C0278061', 'similarity': 1.0, 'semtypes': {'T048'}, 'preferred': 0}]
[{'start': 1, 'end': 22, 'ngram': 'nfluenza like illness', 'term': 'influenza like illness', 'cui': 'C0521839', 'similarity': 0.95, 'semtypes': {'T047'}, 'preferred': 1}]
[{'start': 1, 'end': 17, 'ngram': 'Thrombocytopenia', 'term': 'Thrombocytopenia 3', 'cui': 'C2678311', 'similarity': 0.875, 'semtypes': {'T047'}, 'preferred': 1}, {'start': 1, 'end': 17, 'ngra

In [24]:
len(pool_of_terms), len(valid_fileids)

(245, 180)

In [26]:
", ".join(pool_of_terms)

"Lymphoma, collapse, Bronchiectasis, Sarcoidosis, bleed, sepsis, infiltrates, hyposmolality, Pancreatitis, Withdrawal, respiratory acidosis, septic arthritis, Cerebrovascular Disease, lung collapse, acidosis, pulmonary embolism, Anemia, nsclc, Pneumoperitoneum, Substance abuse, Uterine atony, soft tissue mass, Rib Fractures, Chronic Kidney Disease, Diabetes, Coronary Artery Disease, GI Bleeding, hypoglycemia, K intoxication, bacterial pneumonia, disease, Alcoholism, Systolic Heart Failure, flu, Respiratory Failure, adrenal insufficiency, secondary, hydronephrosis, Pancreatic cancer, hypervolemia, esrd, Esophageal varices, Fungemia, Crohn's, Sepsis, diastolic heart failure, ureteral stone, Coronary artery disease, small cell lung cancer, Cellulitis, Pancytopenia, coronary artery disease, Hyperthyroidism, Rash, pneumonia, Infection, Cirrhosis, prostatitis, Rhabdomyolysis, Infected pacemaker, Macular degeneration, Chronic Renal Failure, Diastolic heart failure, Breast Cancer, anemia, Alte

In [ ]:
pool_of_terms = tuple(pool_of_terms)
pool_of_terms

In [5]:
from pydantic import BaseModel, validator, ValidationError
from typing import List, Set, Literal, Union


class MedicalConditionSelection(BaseModel):
    selected_conditions: List[Union[tuple(Literal[item] for item in pool_of_terms)]]


In [ ]:
MedicalConditionSelection.schema()

In [ ]:
train_df.iloc[4]['Summary2']

In [ ]:
soap_soa_zs = """
You are provided with a patient's medical information from a progress note formatted in the SOAP structure, containing only the Subjective, Objective, and Assessment sections. Your task is to generate a summary that lists the patient's medical problems and diagnoses, including both direct and indirect problems (a past medical problem or consequence from the primary diagnosis). Present your response as a concatenated list of diagnoses separated by semicolons without any additional text or formatting.

Patient Information:
<Subjective>
{subjective_section}
</Subjective> 

<Objective>
{objective_section}
</Objective>

<Assessment>
{assessment_section}
</Assessment>

"""
subject = train_df.iloc[4]['Subjective']
objective = train_df.iloc[4]['Objective']
assessment = train_df.iloc[4]['Assessment']

soap_soa_zs.format(subjective_section=subject, objective_section=objective, assessment_section=assessment)

In [9]:
from openai import OpenAI
import os
from dotenv import load_dotenv
import openai

# Load environment variables from the .env file
load_dotenv()

# Now, you can access the API key using os.getenv
api_key = os.getenv("OPENAI_API_KEY")

if not api_key:
    raise ValueError("OPENAI_API_KEY is not set in the environment variables.")

# Initialize the OpenAI client with the API key
openai.api_key = api_key

client = OpenAI()

In [ ]:
messages = [
    {"role": "system", "content": "You are a medical professional reviewing a progress note to understand a patient's condition."},
    {"role": "user", "content": soap_soa_zs.format(subjective_section=subject, objective_section=objective, assessment_section=assessment)}
    ]


response = client.beta.chat.completions.parse(
            model = "gpt-4o-2024-08-06",
            messages=messages,
            temperature = 0,
            response_format = MedicalConditionSelection
        )


In [ ]:
response